In [1]:
%pip install evaluate dataset transformers datasets accelerate spacy kagglehub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import os
os.environ['WANDB_DISABLED'] = 'true'

import spacy
import string
!python -m spacy download en_core_web_lg
import en_core_web_lg
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     ---------------------------------------- 1.3/400.7 MB 6.5 MB/s eta 0:01:02
      ------------------------------------- 10.0/400.7 MB 25.9 MB/s eta 0:00:16
     - ------------------------------------ 20.2/400.7 MB 33.9 MB/s eta 0:00:12
     -- ----------------------------------- 30.1/400.7 MB 39.4 MB/s eta 0:00:10
     --- ---------------------------------- 34.1/400.7 MB 33.9 MB/s eta 0:00:11
     --- ---------------------------------- 40.6/400.7 MB 36.0 MB/s eta 0:00:10
     ---- --------------------------------- 48.8/400.7 MB 34.3 MB/s eta 0:00:11
     ----- -------------------------------- 53.5/400.7 MB 33.3 MB/s eta 0:00:11
     ----- -------------------------------- 56.9/400.7 MB 31.5 MB/s eta 0:00:11
     ----- -------------------------------- 62.9/400.7 MB 31.0 MB/s eta 0:00:11
     ------ ------------------------------- 68.9/400.7

In [3]:
df = pd.read_csv('C:/newsclassifier/TeamVanga/titles_data_extended.csv', delimiter = ';')
df.columns = ['text', 'label']
df

,text,label
0,Родственник раскрыл настоящую фамилию Пугачёво...,1
1,Предсказания Матроны Московской на 2024-й год:...,1
2,"Пророчество схимонахини Нины об антихристе, ми...",1
3,«Думал об этом»: что Путин сказал о своем прее...,1
4,Путин поручил уведомить россиян об изменениях ...,1
...,...,...
4509,Вопрос времени. Русским в Эстонии установили н...,0
4510,Совет улемов ДУМ принял решение о допустимости...,0
4511,В Кривом Роге мужчина напал с топором на свяще...,0
4512,Главный раввин России поздравил еврейскую общи...,0


In [4]:
nlp = spacy.load("en_core_web_lg")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [5]:
def spacy_tokenizer(text):
    doc = nlp(text)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    text = " ".join(mytokens)
    return text

In [6]:
df['text'] = df['text'].apply(spacy_tokenizer)
df

,text,label
0,родственник раскрыл настоящую фамилию пугачёво...,1
1,предсказания матроны московской на 2024 й год ...,1
2,пророчество схимонахини нины об антихристе мир...,1
3,« думал об этом » что путин сказал о своем пре...,1
4,путин поручил уведомить россиян об изменениях ...,1
...,...,...
4509,вопрос времени русским в эстонии установили но...,0
4510,совет улемов дум принял решение о допустимости...,0
4511,в кривом роге мужчина напал с топором на свяще...,0
4512,главный раввин россии поздравил еврейскую общи...,0


In [7]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4514
})

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
tokenized_df = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 4514/4514 [00:00<00:00, 35835.45 examples/s]


In [11]:
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
metric_f1 = evaluate.load("f1")

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    d = {
        **accuracy.compute(predictions=predictions, references=labels),
        **precision.compute(predictions=predictions, references=labels),
        **metric_f1.compute(predictions=predictions, references=labels)
    }
    return d

In [13]:
id2label = {0: "не кликбейт", 1: "кликбейт"}
label2id = {"не кликбейт": 0, "кликбейт": 1}

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny2", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
split = tokenized_df.train_test_split(test_size = 0.2)
split

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3611
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 903
    })
})

In [16]:
from transformers import TrainingArguments, Trainer

In [17]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    warmup_steps=100,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    remove_unused_columns=True,
    load_best_model_at_end=True,
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split['train'],
    eval_dataset=split['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,F1
1,0.460300,0.267798,0.890365,0.843750,0.830769
2,0.231100,0.213189,0.915836,0.874576,0.871622
3,0.146600,0.199357,0.919158,0.883562,0.876061
4,0.117500,0.213577,0.919158,0.873333,0.877722
5,0.100100,0.219735,0.920266,0.873754,0.879599


TrainOutput(global_step=565, training_loss=0.2179149732125544, metrics={'train_runtime': 97.3505, 'train_samples_per_second': 185.464, 'train_steps_per_second': 5.804, 'total_flos': 5353098154656.0, 'train_loss': 0.2179149732125544, 'epoch': 5.0})

In [18]:
trainer.evaluate(split['test'])

{'eval_loss': 0.19935669004917145,
 'eval_accuracy': 0.9191583610188261,
 'eval_precision': 0.8835616438356164,
 'eval_f1': 0.8760611205432938,
 'eval_runtime': 0.6019,
 'eval_samples_per_second': 1500.22,
 'eval_steps_per_second': 48.18,
 'epoch': 5.0}

In [19]:
news_titles = pd.read_csv('C:/newsclassifier/TeamVanga/news_titles_test.csv', encoding='windows-1251', delimiter=';')
print(news_titles.head())

                                              titles  target  model_predict
0  Фракция Порошенко запустила процедуру отставки...       0              0
1  Установленное при участии Трампа мирное соглаш...       0              0
2         Баффет написал последнее письмо акционерам       0              0
3  Нефтегазовые гиганты в кризисе. Почему бум ИИ ...       0              0
4  Клишас не исключил существование мошеннической...       0              0


In [20]:
test = []

for i in range(len(news_titles['titles'])):
  test.append(news_titles['titles'][i])

test

['Фракция Порошенко запустила процедуру отставки правительства Украины',
 'Установленное при участии Трампа мирное соглашение оказалось под угрозой',
 'Баффет написал последнее письмо акционерам',
 'Нефтегазовые гиганты в кризисе. Почему бум ИИ проходит мимо них',
 'Клишас не исключил существование мошеннической схемы с продажей жилья',
 'Саркози вышел на свободу. Первые кадры после освобождения',
 'Как запрет многократных шенгенских виз повлияет на поездки в ЕС',
 'Что может принести наибольшую доходность до конца года: топ-6 активов',
 'Сколько нужно зарабатывать для большой пенсии в России: расчеты',
 'Курс доллара ЦБ на 11 ноября снизился почти до ?81',
 'ЦБ назвал самый доходный актив в России с начала года',
 'Супругов из Майами признали самой долгоживущей парой в браке',
 'Два аэропорта ввели временные ограничения на полеты',
 '«Баланс» и «право на ошибку»: что известно о новой стратегии «Ростеха»',
 'Литва повторно попросит Белоруссию открыть коридор для застрявших фур',
 'В Бе

In [21]:
from transformers import pipeline

classifier = pipeline("text-classification", model=trainer.model.cpu(), tokenizer=tokenizer)
classifier(test)

Device set to use cpu


[{'label': 'не кликбейт', 'score': 0.984341561794281},
 {'label': 'не кликбейт', 'score': 0.8235191106796265},
 {'label': 'не кликбейт', 'score': 0.9624844193458557},
 {'label': 'не кликбейт', 'score': 0.9781616926193237},
 {'label': 'не кликбейт', 'score': 0.980221152305603},
 {'label': 'не кликбейт', 'score': 0.8501067757606506},
 {'label': 'не кликбейт', 'score': 0.8927252292633057},
 {'label': 'кликбейт', 'score': 0.9164417386054993},
 {'label': 'кликбейт', 'score': 0.9766225814819336},
 {'label': 'не кликбейт', 'score': 0.948674201965332},
 {'label': 'не кликбейт', 'score': 0.8137940764427185},
 {'label': 'не кликбейт', 'score': 0.8244660496711731},
 {'label': 'не кликбейт', 'score': 0.9755392074584961},
 {'label': 'кликбейт', 'score': 0.9020312428474426},
 {'label': 'не кликбейт', 'score': 0.954981803894043},
 {'label': 'не кликбейт', 'score': 0.9870908260345459},
 {'label': 'не кликбейт', 'score': 0.9511603116989136},
 {'label': 'не кликбейт', 'score': 0.8856423497200012},
 {'la